In [1]:
import pandas as pd
import re
import numpy as np
from src import functions
from src import correcciones
import datetime

In [2]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x0000027ED8689BF8; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


In [13]:
# df['canal_venta'] = df.apply(lambda row: correcciones.partner(row), axis=1)
# df['canal_venta'] = df.canal_venta.apply(lambda x: functions.canal(x))

In [93]:
datos = df[['assetid','segmento','Family','product_name_ok','canal_venta','partnername','companyname']]

In [112]:
def billing(cur,df):
    datos = df[['assetid','segmento','Family','product_name_ok','canal_venta','partnername','companyname']]
    cur.execute('''
                select sum(totalamount) as factura, substring(dfrom,0,8) as mes, assetid from billing_details 
                group by mes, assetid
            ''')
    bill_df = pd.DataFrame(cur.fetchall())
    col_names = []
    for e in range(len(cur.description)):
        col_names.append(cur.description[e][0])
    bill_df.rename(columns=dict(zip(list(range(33)), col_names)), inplace=True)
    bill_df['mes']=pd.to_datetime(bill_df.mes, format='%Y-%m', errors='ignore')
    bill_df = bill_df.merge(datos,on ='assetid',how='left')
    bill_df = bill_df[bill_df['segmento'] != '1-GRATIS']
    return bill_df

In [113]:
bill_df = billing(cur,df)

In [114]:
bill_df.segmento.unique()

array(['3-NOTRIAL', nan, '2-TRIAL', '4-MIGRADO', '5-MIGRADO GRATIS'],
      dtype=object)

In [124]:
marco = bill_df[(bill_df['segmento']=='5-MIGRADO GRATIS')&(bill_df['factura']!=0.0)]
marco.to_csv('./migrados_gratis_factura.csv',index=False)

In [127]:
bill_df[bill_df['assetid']=='02i0N00000Mh0FUQAZ']

,factura,mes,assetid,segmento,Family,product_name_ok,canal_venta,partnername,companyname
1087,12.00,2019-12-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
1772,12.00,2019-08-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
2187,12.00,2020-02-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
3080,12.00,2019-09-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
3232,12.00,2019-10-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
4462,0.00,2019-06-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
6145,5.62,2019-07-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
7273,12.00,2019-11-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN
7604,12.00,2020-01-01,02i0N00000Mh0FUQAZ,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
df[df['assetid']=='02i0N00000Mh0FUQAZ']

,segmento,Family,product_name,companyname,sitename,cod_postal,created_as_trial,purchase_date,cancellation_date,service_date,...,Segmento_Empleados,assetid,asset_status,migrado,assetpadre_mig,assethijo_mig,rfb_migration,date_migration,purchase_migration,product_name_ok


In [69]:
bill_df.dtypes

factura                   float64
mes                datetime64[ns]
assetid                    object
segmento                   object
Family                     object
product_name_ok            object
canal_venta                object
dtype: object

In [96]:
bill_df[['factura']].groupby(by=[bill_df.segmento,bill_df.Family,bill_df.product_name_ok,bill_df.canal_venta,bill_df.mes]).sum()

factura
segmento         Family    product_name_ok canal_venta   mes                
1-GRATIS         X MUSIC F X Music         WEB           2018-11-01     0.00
                                                         2018-12-01     0.00
                                                         2019-01-01    38.70
                                                         2019-02-01    75.00
                                                         2019-03-01    70.15
...                                                                      ...
4-MIGRADO        X UCOM    Teamwork+PBX    DHO           2019-11-01     5.59
                                                         2020-01-01     2.28
                                           IT integrator 2019-05-01     9.68
                                                         2019-06-01    10.80
5-MIGRADO GRATIS X UCOM    Teamwork+PBX    WEB           2019-05-01     0.64

[521 rows x 1 columns]

In [4]:
df = functions.base(cur)

In [8]:
# df.to_csv('../df.csv ',encoding='CP1252')

In [17]:
# df_altas.to_csv('../df_altas.csv ',encoding='CP1252')

In [5]:
df_altas = functions.base_altas(df)

In [9]:
df_altas.head()

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isClient,isActive,assets_dea,isBaja
0,03817373V,WEB,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,1,0,2,1
1,07477239P,DHO,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,1,0,1,1
2,11058321J,DHO,1,2019-01-23 00:00:00,NaT,1,1,1,0,0
3,12371541W,WEB,0,2018-12-18 00:00:01,2019-01-02 12:15:09,1,1,0,1,1
4,16601899Q,DHO,2,2019-05-22 00:00:01,NaT,6,1,1,4,0


In [4]:
def base_altas_vis(df):
#     df = df[df['rfb__c'].notnull()]
    assets = df.isAsset.groupby(
        by=[df.cif, df.canal_venta]).sum().reset_index()
    assets.rename(columns={'isAsset': 'assets'}, inplace=True)
    assets_act = df.isActive.groupby(
        by=[df.cif, df.canal_venta]).sum().reset_index()
    assets_act.rename(columns={'isActive': 'assets_act'}, inplace=True)
    altas = df.rfb__c.groupby(by=[df.cif, df.canal_venta]).min().reset_index()
    bajas = df.deactivation_date.groupby(
        by=[df.cif, df.canal_venta]).max().reset_index()
    df_altas = pd.merge(pd.merge(pd.merge(assets_act, altas, on=['cif', 'canal_venta']), bajas, on=[
                        'cif', 'canal_venta']), assets, on=['cif', 'canal_venta'])
    df_altas['isClient'] = 1
    df_altas['isActive'] = df_altas.assets_act.apply(
        lambda x: 1 if x >= 1 else 0)
    df_altas['deactivation_date'] = df_altas.apply(
        lambda row: deactivation(row), axis=1)
    df_altas['assets_dea'] = df_altas['assets'] - df_altas['assets_act']
    df_altas['isBaja'] = -(df_altas['isActive']-1)
    df_altas.rename(columns={'assets': 'assets_tot',
                             'rfb__c': 'rfb',
                             }, inplace=True)
    return df_altas

## Ventas

In [6]:
functions.ass_vta_mes(df,2)
# a.iloc[a.index.get_level_values('Family') == 'X UCOM']

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
canal_venta,,,,,,,,,,,,,,,,,
DHO,2,14,7,28,53,27,19,45,104,92,17,103,117,100,113,119,7
IT integrator,0,1,14,6,1,2,12,9,22,6,1,47,38,5,1,11,7
Venta Directa,0,5,1,0,0,1,2,0,1,1,0,4,6,2,15,6,8
WEB,1,4,12,1,2,24,8,8,11,6,5,12,12,3,2,0,0


In [8]:
functions.cli_altas_mes_agg(df_altas,tipo=1)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
canal_venta,,,,,,,,,,,,,,,,,
DHO,2,2,9,7,19,8,8,13,29,30,19,28,35,34,24,35,4
IT integrator,0,1,3,3,0,1,6,2,3,2,0,5,0,1,0,0,0
Venta Directa,0,1,0,1,0,0,1,0,1,1,0,2,2,2,1,4,1
WEB,1,1,6,2,1,5,3,6,3,3,2,5,7,2,0,0,0


In [16]:
x = df[df['Family']=='X UCOM']
x.to_csv('../x.csv ',encoding='CP1252')

In [4]:
# functions.ventas(df)

In [5]:
# functions.altas(df,df_altas )

In [5]:
functions.bajas(df,df_altas)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
index,,,,,,,,,,,,,,,,,
isBaja,0,0,1,3,0,2,6,8,5,4,5,8,10,7,7,8,1
DHO,0,0,0,1,0,2,4,4,2,3,2,3,5,4,6,8,0
IT integrator,0,0,0,0,0,0,0,2,1,0,0,3,0,0,0,0,0
WEB,0,0,1,2,0,0,2,2,2,1,3,2,5,3,1,0,1
DHO_X FIBER,0,0,0,0,0,1,0,0,0,2,1,1,1,0,1,2,0
DHO_X MUSIC F,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0
DHO_X PRIVACY,0,0,0,0,0,0,2,3,1,1,2,1,3,0,0,0,0
DHO_X PROTECTION F,0,0,0,0,0,0,0,2,3,0,1,1,1,2,2,6,0
DHO_X SECURITY,0,0,0,0,0,0,0,0,0,1,0,0,2,0,2,0,0


In [6]:
functions.cli_bajas_mes(df, 3)

2018-10  2018-11  2018-12  2019-01  2019-02  \
canal_venta   Family                                                        
DHO           X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        0        1        0   
              X PRIVACY             0        0        0        0        0   
              X PROTECTION F        0        0        0        0        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        0        1        1   
IT integrator X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        0        0        0   
              X PROTECTION F        0        0        0        0        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        0        0        0   
WEB           X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        1        0        0   
              X PRIVACY             0        0        0        0        0   
              X PROTECTION F        0        0        0        2        0   
              X UCOM                0        0        0        0        1   

                              2019-03  2019-04  2019-05  2019-06  2019-07  \
canal_venta   Family                                                        
DHO           X FIBER               1        0        0        0        2   
              X MUSIC F             1        1        0        0        0   
              X PRIVACY             0        2        3        1        1   
              X PROTECTION F        0        0        2        3        0   
              X SECURITY            0        0        0        0        1   
              X UCOM                2        1        1        2        4   
IT integrator X FIBER               0        0        0        0        0   
              X MUSIC F             0        0        1        0        0   
              X PROTECTION F        0        1        0        1        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        1        0        0   
WEB           X FIBER               1        0        0        1        0   
              X MUSIC F             0        1        1        0        0   
              X PRIVACY             0        1        0        0        0   
              X PROTECTION F        0        1        1        0        0   
              X UCOM                0        0        0        2        1   

                              2019-08  2019-09  2019-10  2019-11  2019-12  \
canal_venta   Family                                                        
DHO           X FIBER               1        1        1        0        1   
              X MUSIC F             0        0        0        0        0   
              X PRIVACY             2        1        3        0        0   
              X PROTECTION F        1        1        1        2        2   
              X SECURITY            0        0        2        0        2   
              X UCOM                3        2        4        3        5   
IT integrator X FIBER               0        2        0        0        1   
              X MUSIC F             0        1        0        0        0   
              X PROTECTION F        0        1        0        1        0   
              X SECURITY            0        0        0        0        0   
              X UCOM                0        0        0        0        0   
WEB           X FIBER               0        0        1        0        0   
              X MUSIC F             1        0        0        0        0   
              X PRIVACY             0        0        0        0        0   
              X PROTECTI

In [5]:
mig_df=functions.migBase(cur)

In [11]:
# mig_df

In [6]:
migin, migout = functions.mig_producto_mes(mig_df,df)

In [25]:
migin.reset_index(inplace=True)
migin['index'] = migin['canal_venta']+'_'+migin['product_name']
migin.drop(columns=['canal_venta','Family','product_name'],inplace=True)
migin.set_index(['index'],inplace=True)

In [7]:
migin

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        0        0        0   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        1        0   
              X UCOM  Teamwork+PBX              0        0        0       11   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              0        2        1        1   
IT integrator X UCOM  Teamwork+PBX              1        0        0        0   

                                          2019-10  2019-11  2019-12  2020-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba        0        0        0        0   
              X UCOM  Teamwork+PBX              3        1        0        1   
IT integrator X UCOM  Teamwork+PBX              0        0        0        0   

                                          2020-02  
canal_venta   Family  product_name                 
DHO           X FIBER X Fiber 100 - neba        0  
              X UCOM  Teamwork+PBX              0  
IT integrator X UCOM  Teamwork+PBX              0

In [33]:
getVelocityFrom('20')

'Portable 20'